# TradingAgents: Unrolled LangGraph Notebook

This notebook demonstrates the TradingAgents multi-agent system with **unrolled graph execution**.

Instead of using LangGraph's automatic execution, we manually execute each node step-by-step:
- **Analyst nodes** (market, social, news, fundamentals)
- **Research debate** (bull vs bear researchers)
- **Trader node**
- **Risk debate** (risky, safe, neutral analysts)
- **Portfolio manager decision**

This allows you to:
- Run one node at a time
- Inspect state between each step
- Debug individual agents
- Modify the flow manually

## 1. Setup and Imports

In [1]:
import os
import sys
from pathlib import Path
from typing import Dict, Any
from datetime import datetime

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from langgraph.graph.state import _pick_mapper
from langgraph.pregel._algo import _proc_input
from langgraph.pregel._read import PregelNode
from langgraph.graph import StateGraph

from tradingagents.agents.utils.agent_states import AgentState, InvestDebateState, RiskDebateState
from tradingagents.default_config import DEFAULT_CONFIG
from tradingagents.graph.conditional_logic import ConditionalLogic

print("✓ Imports successful")

✓ Imports successful


## 2. Configure API Keys

In [2]:
from getpass import getpass

# Check OpenAI API key
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("🔑 Enter your OpenAI API key: ")
else:
    print("✓ OpenAI API key found")

# Check Finnhub API key
if not os.getenv("FINNHUB_API_KEY"):
    os.environ["FINNHUB_API_KEY"] = getpass("🔑 Enter your Finnhub API key: ")
else:
    print("✓ Finnhub API key found")

✓ OpenAI API key found
✓ Finnhub API key found


## 3. Create State Executor

This class manages LangGraph state channels without automatic execution.

In [3]:
class TradingAgentsExecutor:
    """Manual executor for TradingAgents graph using official LangGraph channels."""
    
    def __init__(self):
        # Create StateGraph to get official channels
        temp_builder = StateGraph(AgentState)
        temp_builder._add_schema(AgentState)
        
        # Get real channels from StateGraph
        self.channels = temp_builder.channels.copy()
        
        # Initialize all channels
        for channel in self.channels.values():
            channel.update([])
        
        # Get official mapper for AgentState
        state_keys = list(AgentState.__annotations__.keys())
        self.mapper = _pick_mapper(state_keys, AgentState)
        
        # Create PregelNode wrappers
        self.nodes = {}
    
    def register_node(self, node_name: str):
        """Register a node for execution."""
        state_keys = list(AgentState.__annotations__.keys())
        self.nodes[node_name] = PregelNode(
            tags=[node_name],
            triggers=state_keys,
            channels=state_keys,
            writers=[],
            mapper=self.mapper,
            retry_policy=None,
            cache_policy=None
        )
    
    def create_node_input(self, node_name: str) -> Dict[str, Any]:
        """Create input for a specific node using official _proc_input."""
        if node_name not in self.nodes:
            self.register_node(node_name)
        
        node = self.nodes[node_name]
        return _proc_input(
            proc=node,
            managed={},
            channels=self.channels,
            for_execution=True,
            scratchpad=None,
            input_cache=None
        )
    
    def update_state(self, node_output: Dict[str, Any]):
        """Update channels with node output."""
        if isinstance(node_output, dict):
            for key, value in node_output.items():
                if key in self.channels:
                    channel = self.channels[key]
                    channel_type = type(channel).__name__
                    
                    if channel_type == "LastValue":
                        channel.update([value])
                    elif channel_type == "BinaryOperatorAggregate":
                        if isinstance(value, list):
                            channel.update(value)
                        else:
                            channel.update([value])
                    else:
                        if isinstance(value, list):
                            channel.update(value)
                        else:
                            channel.update([value])
    
    def get_state(self) -> Dict[str, Any]:
        """Get current state from all channels."""
        state = {}
        for name, channel in self.channels.items():
            if channel.is_available():
                state[name] = channel.get()
        return state

# Create executor
executor = TradingAgentsExecutor()
print("✓ State executor created")

✓ State executor created


## 4. Initialize TradingAgents Components

In [4]:
from tradingagents.agents.utils.agent_utils import Toolkit
from tradingagents.agents.utils.memory import FinancialSituationMemory
from tradingagents.dataflows.interface import set_config
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode

# Configuration
config = DEFAULT_CONFIG.copy()
config["deep_think_llm"] = "o1-mini"  # Cost-effective model
config["quick_think_llm"] = "gpt-4o-mini"
config["max_debate_rounds"] = 2
config["max_risk_discuss_rounds"] = 2
config["online_tools"] = True

# Set global config for interface
set_config(config)

# Initialize LLMs
quick_llm = ChatOpenAI(model=config["quick_think_llm"], temperature=0)
deep_llm = ChatOpenAI(model=config["deep_think_llm"], temperature=1)

# Initialize toolkit with config
toolkit = Toolkit(config=config)

# Initialize memories with config
bull_memory = FinancialSituationMemory("bull_memory", config)
bear_memory = FinancialSituationMemory("bear_memory", config)
trader_memory = FinancialSituationMemory("trader_memory", config)
research_memory = FinancialSituationMemory("research_manager_memory", config)
risk_memory = FinancialSituationMemory("risk_manager_memory", config)

# Initialize conditional logic
conditional_logic = ConditionalLogic(
    max_debate_rounds=config["max_debate_rounds"],
    max_risk_discuss_rounds=config["max_risk_discuss_rounds"]
)

print("✓ Components initialized")

✓ Components initialized


## 5. Create Agent Nodes

In [5]:
from tradingagents.agents import (
    create_market_analyst,
    create_social_media_analyst,
    create_news_analyst,
    create_fundamentals_analyst,
    create_bull_researcher,
    create_bear_researcher,
    create_research_manager,
    create_trader,
    create_risky_debator,
    create_neutral_debator,
    create_safe_debator,
    create_risk_manager
)

# Create analyst nodes
market_analyst = create_market_analyst(quick_llm, toolkit)
social_analyst = create_social_media_analyst(quick_llm, toolkit)
news_analyst = create_news_analyst(quick_llm, toolkit)
fundamentals_analyst = create_fundamentals_analyst(quick_llm, toolkit)

# Create researcher nodes
bull_researcher = create_bull_researcher(quick_llm, bull_memory)
bear_researcher = create_bear_researcher(quick_llm, bear_memory)
research_manager = create_research_manager(deep_llm, research_memory)

# Create trader node
trader = create_trader(quick_llm, trader_memory)

# Create risk nodes
risky_analyst = create_risky_debator(quick_llm)
neutral_analyst = create_neutral_debator(quick_llm)
safe_analyst = create_safe_debator(quick_llm)
risk_manager = create_risk_manager(deep_llm, risk_memory)

# Create tool nodes with toolkit methods
market_tools = ToolNode([
    toolkit.get_YFin_data_online,
    toolkit.get_stockstats_indicators_report_online,
    toolkit.get_YFin_data,
    toolkit.get_stockstats_indicators_report,
])

social_tools = ToolNode([
    toolkit.get_stock_news_openai,
    toolkit.get_reddit_stock_info,
])

news_tools = ToolNode([
    toolkit.get_global_news_openai,
    toolkit.get_google_news,
    toolkit.get_finnhub_news,
    toolkit.get_reddit_news,
])

fundamentals_tools = ToolNode([
    toolkit.get_fundamentals_openai,
    toolkit.get_finnhub_company_insider_sentiment,
    toolkit.get_finnhub_company_insider_transactions,
    toolkit.get_simfin_balance_sheet,
    toolkit.get_simfin_cashflow,
    toolkit.get_simfin_income_stmt,
])

print("✓ All agent nodes created")

✓ All agent nodes created


## 6. Initialize State

Set the ticker and date for analysis.

In [6]:
# Set analysis parameters
TICKER = "NVDA"
DATE = "2024-05-10"

# Initialize state
initial_state = {
    "company_of_interest": TICKER,
    "trade_date": DATE,
    "messages": [],
    "sender": "user",
    "market_report": "",
    "sentiment_report": "",
    "news_report": "",
    "fundamentals_report": "",
    "investment_debate_state": {
        "bull_history": "",
        "bear_history": "",
        "history": "",
        "current_response": "",
        "judge_decision": "",
        "count": 0
    },
    "investment_plan": "",
    "trader_investment_plan": "",
    "risk_debate_state": {
        "risky_history": "",
        "safe_history": "",
        "neutral_history": "",
        "history": "",
        "latest_speaker": "",
        "current_risky_response": "",
        "current_safe_response": "",
        "current_neutral_response": "",
        "judge_decision": "",
        "count": 0
    },
    "final_trade_decision": ""
}

# Update executor state
executor.update_state(initial_state)

print(f"✓ State initialized for {TICKER} on {DATE}")

✓ State initialized for NVDA on 2024-05-10


## 7. Execute Analyst Nodes

### Helper Function for Tool Execution

In [7]:
def run_analyst_with_tools(analyst_name, analyst_func, tool_node, executor):
    """
    Run an analyst node with tool execution loop.
    
    This handles:
    1. Running the analyst
    2. Executing any tool calls
    3. Running the analyst again with tool results
    4. Clearing messages after completion
    """
    from langchain_core.messages import RemoveMessage, HumanMessage
    
    # Get current state
    state = executor.create_node_input(analyst_name)
    print(f"Running {analyst_name.replace('_', ' ').title()}...")
    
    # Execute analyst
    result = analyst_func(state)
    executor.update_state(result)
    
    # Check if tools need to be called
    current_state = executor.get_state()
    if current_state['messages'] and hasattr(current_state['messages'][-1], 'tool_calls'):
        if current_state['messages'][-1].tool_calls:
            print(f"  → Calling {analyst_name} tools...")
            tool_result = tool_node.invoke(current_state)
            executor.update_state(tool_result)
            
            # Run analyst again with tool results
            state = executor.create_node_input(analyst_name)
            result = analyst_func(state)
            executor.update_state(result)
    
    # Clear messages (simulating msg_delete node)
    current_state = executor.get_state()
    if current_state['messages']:
        messages = current_state['messages']
        removal_operations = [RemoveMessage(id=m.id) for m in messages]
        placeholder = HumanMessage(content="Continue")
        executor.update_state({"messages": removal_operations + [placeholder]})
    
    return executor.get_state()

print("✓ Helper function defined")

✓ Helper function defined


### 7.1 Market Analyst

In [8]:
state = run_analyst_with_tools("market_analyst", market_analyst, market_tools, executor)

print("\n" + "="*60)
print("MARKET ANALYST REPORT")
print("="*60)
print(state.get('market_report', 'No report generated'))
print("="*60)

Running Market Analyst...
  → Calling market_analyst tools...

MARKET ANALYST REPORT



### 7.2 Social Media Analyst

In [9]:
state = run_analyst_with_tools("social_analyst", social_analyst, social_tools, executor)

print("\n" + "="*60)
print("SOCIAL MEDIA ANALYST REPORT")
print("="*60)
print(state.get('sentiment_report', 'No report generated'))
print("="*60)

Running Social Analyst...
  → Calling social_analyst tools...

SOCIAL MEDIA ANALYST REPORT
### Comprehensive Analysis of NVIDIA Corporation (NVDA) - May 2024

#### Overview
NVIDIA Corporation (NVDA) has been a focal point in the stock market and social media discussions over the past week, primarily driven by its stock performance and significant financial announcements. This report synthesizes recent news, social media sentiment, and stock market activity to provide insights for traders and investors.

#### Stock Performance
- **Current Price**: As of May 10, 2024, NVDA's stock is priced at **$188.44**, with a slight decrease of **$0.45** (-0.00%) from the previous close.
- **Price Movement**: The stock has shown notable fluctuations, closing at **$86.37** on April 30, 2024, and rising to **$90.52** by May 7, 2024. This upward trend continued, with the stock closing at **$88.72** on May 9, 2024, before stabilizing around the current price.
- **Trading Volume**: The intraday volume on 

### 7.3 News Analyst

In [10]:
state = run_analyst_with_tools("news_analyst", news_analyst, news_tools, executor)

print("\n" + "="*60)
print("NEWS ANALYST REPORT")
print("="*60)
print(state.get('news_report', 'No report generated'))
print("="*60)

Running News Analyst...
  → Calling news_analyst tools...

NEWS ANALYST REPORT
### Current State of the World Relevant for Trading and Macroeconomics (May 10, 2024)

#### Macroeconomic Developments

1. **U.S. Labor Market Data (May 3, 2024)**:
   - The U.S. Department of Labor reported that 175,000 jobs were added in April, significantly below the expected 243,000. This indicates a cooling labor market, which may influence the Federal Reserve's monetary policy decisions moving forward. Wage growth was also lower than anticipated, and the unemployment rate increased slightly.

2. **U.S. Stock Market Performance**:
   - Following the labor market report, U.S. equities saw a rally, with the S&P 500 Index reaching a one-month high. The weaker job data has led to speculation about potential Federal Reserve rate cuts, particularly benefiting sectors like real estate and energy.

3. **Crude Oil Inventory Report**:
   - The Energy Information Administration (EIA) reported a decrease of 1.4 mil

### 7.4 Fundamentals Analyst

In [11]:
state = run_analyst_with_tools("fundamentals_analyst", fundamentals_analyst, fundamentals_tools, executor)

print("\n" + "="*60)
print("FUNDAMENTALS ANALYST REPORT")
print("="*60)
print(state.get('fundamentals_report', 'No report generated'))
print("="*60)

Running Fundamentals Analyst...
  → Calling fundamentals_analyst tools...

FUNDAMENTALS ANALYST REPORT
It seems that I was unable to retrieve the latest fundamental information for NVIDIA Corporation (NVDA) specifically for the week leading up to May 10, 2024. However, I can provide a summary based on the latest available data and general insights into NVIDIA's financial health and market position.

### NVIDIA Corporation (NVDA) Overview

**Company Profile:**
- **Industry:** Semiconductors
- **Headquarters:** Santa Clara, California, USA
- **Founded:** 1993
- **CEO:** Jensen Huang
- **Core Business:** NVIDIA is a leading designer of graphics processing units (GPUs) for gaming and professional markets, as well as system on a chip units (SoCs) for the mobile computing and automotive market.

### Recent Financial Metrics (as of October 3, 2025)
- **Current Stock Price:** $188.185
- **Change:** -$0.70 (-0.00%)
- **Open Price:** $189.20
- **Intraday Volume:** 61,704,306 shares
- **Intraday 

## 9. Research Manager Decision

In [12]:
state = executor.create_node_input("research_manager")
print("Running Research Manager...\n")

result = research_manager(state)
executor.update_state(result)

current_state = executor.get_state()
print("="*60)
print("RESEARCH MANAGER SYNTHESIS")
print("="*60)
print(current_state.get('investment_plan', 'No plan generated'))
print("="*60)

Running Research Manager...

RESEARCH MANAGER SYNTHESIS
It looks like the sections for your past reflections on mistakes and the debate history are currently empty. To provide a comprehensive evaluation and develop a detailed investment plan, I need the following information:

1. **Past Reflections on Mistakes:** Insights or lessons you've learned from previous investment decisions that didn't go as planned. This helps in refining the decision-making process.

2. **Debate History:** Details of the recent debate between the bear and bull analysts, including their main arguments, evidence presented, and any supporting data.

Could you please provide these details? Once I have that information, I can effectively summarize the key points, make a clear recommendation, and develop a strategic investment plan tailored to your needs.


## 10. Trader Investment Plan

In [13]:
state = executor.create_node_input("trader")
print("Running Trader...\n")

result = trader(state)
executor.update_state(result)

current_state = executor.get_state()
print("="*60)
print("TRADER INVESTMENT PLAN")
print("="*60)
print(current_state.get('trader_investment_plan', 'No plan generated'))
print("="*60)

Running Trader...

TRADER INVESTMENT PLAN
Based on the proposed investment plan for NVDA, I will analyze the current market conditions, technical indicators, macroeconomic factors, and social media sentiment to formulate a recommendation.

1. **Technical Market Trends:** NVDA has shown a strong upward trend in its stock price, supported by robust earnings reports and positive guidance. Key technical indicators, such as moving averages and RSI, suggest that the stock is in a bullish phase, although it may be approaching overbought territory.

2. **Macroeconomic Indicators:** The semiconductor industry is experiencing growth due to increased demand for AI and machine learning technologies. However, potential headwinds include supply chain disruptions and inflationary pressures that could impact margins.

3. **Social Media Sentiment:** Sentiment analysis indicates a predominantly positive outlook among retail investors, with many expressing confidence in NVDA's long-term growth potential.

## 11. Execute Risk Debate

Risky, Safe, and Neutral analysts debate risk levels.

In [14]:
print("Starting Risk Analysis Debate...\n")

# Start with Risky Analyst
current_speaker = "risky"
risk_round = 0

while risk_round < config["max_risk_discuss_rounds"]:
    risk_round += 1
    print(f"\n{'='*60}")
    print(f"RISK ROUND {risk_round}: {current_speaker.upper()} ANALYST")
    print(f"{'='*60}")
    
    state = executor.create_node_input(f"{current_speaker}_analyst")
    
    if current_speaker == "risky":
        result = risky_analyst(state)
        next_node = conditional_logic.should_continue_risk_analysis(result)
    elif current_speaker == "safe":
        result = safe_analyst(state)
        next_node = conditional_logic.should_continue_risk_analysis(result)
    else:  # neutral
        result = neutral_analyst(state)
        next_node = conditional_logic.should_continue_risk_analysis(result)
    
    executor.update_state(result)
    
    # Display response
    current_state = executor.get_state()
    risk_state = current_state['risk_debate_state']
    
    if current_speaker == "risky":
        print(risk_state['current_risky_response'])
    elif current_speaker == "safe":
        print(risk_state['current_safe_response'])
    else:
        print(risk_state['current_neutral_response'])
    
    # Check if should continue debate
    if next_node == "Risk Judge":
        print("\n→ Moving to Risk Judge")
        break
    
    # Rotate speakers: risky -> safe -> neutral -> risky
    if current_speaker == "risky":
        current_speaker = "safe"
    elif current_speaker == "safe":
        current_speaker = "neutral"
    else:
        current_speaker = "risky"

print(f"\n{'='*60}")
print("Risk debate complete")
print(f"{'='*60}")

Starting Risk Analysis Debate...


RISK ROUND 1: RISKY ANALYST
Risky Analyst: Let's dive into the arguments presented by the conservative and neutral analysts regarding the NVDA investment decision. 

First, the conservative analyst often emphasizes caution, particularly around the potential for market corrections and the risks associated with supply chain disruptions and inflationary pressures. While these concerns are valid, they can also lead to missed opportunities. The semiconductor industry is not just recovering; it is on the brink of a transformative boom driven by AI and machine learning. NVDA is at the forefront of this revolution, and the demand for its products is only expected to grow. The recent announcement of the R100 AI chip, which utilizes cutting-edge 3nm technology, is a game-changer. This innovation positions NVDA to capture significant market share and revenue growth, far outweighing the risks posed by temporary supply chain issues.

Moreover, the conservative sta

## 12. Risk Manager Final Decision

In [15]:
state = executor.create_node_input("risk_manager")
print("Running Risk Manager (Portfolio Manager)...\n")

result = risk_manager(state)
executor.update_state(result)

current_state = executor.get_state()
print("="*60)
print("FINAL TRADE DECISION")
print("="*60)
print(current_state.get('final_trade_decision', 'No decision generated'))
print("="*60)

Running Risk Manager (Portfolio Manager)...

FINAL TRADE DECISION
**Recommendation: Buy**

---

### **Summary of Key Arguments**

**Risky Analyst:**
- **Innovation Leadership:** NVDA is at the forefront of the AI and machine learning revolution, particularly highlighted by the recent launch of the R100 AI chip utilizing cutting-edge 3nm technology.
- **Market Growth Potential:** The semiconductor industry is not just recovering but poised for a transformative boom, with NVDA positioned to capture significant market share and revenue growth.
- **Favorable Macroeconomic Conditions:** Anticipated Federal Reserve rate cuts due to a cooling labor market could benefit growth stocks like NVDA by making borrowing cheaper and stimulating technology investments.
- **Positive Market Sentiment:** High trading volumes and strong retail investor confidence indicate robust market interest and potential for substantial gains.

**Safe/Conservative Analyst:**
- **Market Volatility Risks:** The semicondu

## 13. Summary and Analysis

In [16]:
final_state = executor.get_state()

print(f"\n{'='*80}")
print(f"COMPLETE TRADING ANALYSIS FOR {TICKER} ON {DATE}")
print(f"{'='*80}\n")

print("📊 ANALYST REPORTS:")
print(f"  - Market Report: {len(final_state.get('market_report', ''))} chars")
print(f"  - Sentiment Report: {len(final_state.get('sentiment_report', ''))} chars")
print(f"  - News Report: {len(final_state.get('news_report', ''))} chars")
print(f"  - Fundamentals Report: {len(final_state.get('fundamentals_report', ''))} chars")

print("\n🎯 RESEARCH DEBATE:")
debate_state = final_state.get('investment_debate_state', {})
print(f"  - Debate Rounds: {debate_state.get('count', 0)}")
print(f"  - Bull Arguments: {len(debate_state.get('bull_history', ''))} chars")
print(f"  - Bear Arguments: {len(debate_state.get('bear_history', ''))} chars")

print("\n💼 INVESTMENT PLAN:")
print(f"  - Research Manager Plan: {len(final_state.get('investment_plan', ''))} chars")
print(f"  - Trader Plan: {len(final_state.get('trader_investment_plan', ''))} chars")

print("\n⚠️ RISK ANALYSIS:")
risk_state = final_state.get('risk_debate_state', {})
print(f"  - Risk Rounds: {risk_state.get('count', 0)}")
print(f"  - Risky Analysis: {len(risk_state.get('risky_history', ''))} chars")
print(f"  - Safe Analysis: {len(risk_state.get('safe_history', ''))} chars")
print(f"  - Neutral Analysis: {len(risk_state.get('neutral_history', ''))} chars")

print("\n✅ FINAL DECISION:")
print(f"  {final_state.get('final_trade_decision', 'No decision')[:200]}...")

print(f"\n{'='*80}")


COMPLETE TRADING ANALYSIS FOR NVDA ON 2024-05-10

📊 ANALYST REPORTS:
  - Market Report: 0 chars
  - Sentiment Report: 4753 chars
  - News Report: 5124 chars
  - Fundamentals Report: 3454 chars

🎯 RESEARCH DEBATE:
  - Debate Rounds: 0
  - Bull Arguments: 0 chars
  - Bear Arguments: 0 chars

💼 INVESTMENT PLAN:
  - Research Manager Plan: 781 chars
  - Trader Plan: 1291 chars

⚠️ RISK ANALYSIS:
  - Risk Rounds: 2
  - Risky Analysis: 2977 chars
  - Safe Analysis: 2832 chars
  - Neutral Analysis: 0 chars

✅ FINAL DECISION:
  **Recommendation: Buy**

---

### **Summary of Key Arguments**

**Risky Analyst:**
- **Innovation Leadership:** NVDA is at the forefront of the AI and machine learning revolution, particularly highlig...

